In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import copy
import os
from tqdm import tqdm

In [2]:
dataPath = "./Data/"
patchesDataFile = "NMS_patch_notes.xlsx"
df = pd.read_excel(dataPath+patchesDataFile,index_col=0)

In [3]:
print(df.head(),"\n"*2,"="*20)
print(df.describe(),"\n"*2,"="*20)
print(df.info(),"\n"*2,"="*20)

           Title       Date  \
0  PC Patch 1.04 2016-08-18   
1  PC Patch 1.05 2016-08-19   
2  PC Patch 1.06 2016-08-20   
3  PC Patch 1.07 2016-09-02   
4  PC Patch 1.08 2016-09-04   

                                               Notes  Patch update  \
0  "PC Patch 1.04 - No Man's Sky"\n'Menu'\n'Lates...         False   
1  "PC Patch 1.05 - No Man's Sky"\n'Menu'\n'Lates...         False   
2  "PC Patch 1.06 - No Man's Sky"\n'Menu'\n'Lates...         False   
3  "PC Patch 1.07 - No Man's Sky"\n'Menu'\n'Lates...         False   
4  "PC Patch 1.08 - No Man's Sky"\n'Menu'\n'Lates...         False   

   Bug fix update  Scrape error                             Patch page url  
0            True         False  https://www.nomanssky.com//pc-patch-1-04/  
1            True         False  https://www.nomanssky.com//pc-patch-1-05/  
2            True         False  https://www.nomanssky.com//pc-patch-1-06/  
3            True         False  https://www.nomanssky.com//pc-patch-1-07/  
4      

In [4]:
steamDBdataPath = "../SteamDB_Data_Downloads/"
steamDBDataFile = "NMS_patch-release-page-grab_steamdb-info.xlsx"
sdbdf = pd.read_excel(steamDBdataPath+steamDBDataFile,index_col=0)
sdbdf = sdbdf.reset_index()

In [5]:
print(sdbdf.head(),"\n"*2,"="*20)
print(sdbdf.describe(),"\n"*2,"="*20)
print(sdbdf.info(),"\n"*2,"="*20)

        Date  Day      Time                              Patch Title  \
0 2025-02-14  Fri  19:56:00                                 No title   
1 2025-02-14  Fri  14:31:00                                 No title   
2 2025-02-13  Thu  11:41:00                                 No title   
3 2025-02-12  Wed  12:38:00   MAJOR Expedition 17: "Titan" Now Live!   
4 2025-02-12  Wed  12:37:00                                 No title   

   Unnamed: 4  Unnamed: 5   BuildID  
0         NaN         NaN  17371496  
1         NaN         NaN  17367982  
2         NaN         NaN  17351003  
3         NaN         NaN  17336353  
4         NaN         NaN  17336339   

                                Date  Unnamed: 4  Unnamed: 5       BuildID
count                            280         0.0         0.0  2.800000e+02
mean   2021-12-09 23:54:51.428571392         NaN         NaN  9.044814e+06
min              2016-08-12 00:00:00         NaN         NaN  1.276924e+06
25%              2020-01-12 18:00:00 

In [6]:
sdbdf = sdbdf.drop(["Day","Time","Unnamed: 4","Unnamed: 5"],axis=1)
sdbdf = sdbdf.astype({'Date':"datetime64[ns]"})
sdbdf = sdbdf.replace(to_replace={"Patch Title":"No title"}, value = pd.NA)

In [7]:
sdbdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         280 non-null    datetime64[ns]
 1   Patch Title  12 non-null     object        
 2   BuildID      280 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 6.7+ KB


In [8]:
print(f"df: {len(pd.unique(df["Date"]))} unique dates out of {len(df["Date"])}")
print(f"sdbdf: {len(pd.unique(sdbdf["Date"]))} unique dates out of {len(sdbdf["Date"])}")

df: 192 unique dates out of 239
sdbdf: 221 unique dates out of 280


In [9]:
combidf = df.merge(sdbdf,how="outer",on="Date")
combidf

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url,Patch Title,BuildID
0,foundation-update,NaT,'Patch notes'\n'General Gameplay'\n'Fixed tech...,True,False,False,https://www.nomanssky.com//foundation-update,NaN,NaN
1,pathfinder-update,NaT,'Patch Notes'\n'Audio'\n'50% more generative s...,True,False,False,https://www.nomanssky.com//pathfinder-update,NaN,NaN
2,atlas-rises-update,NaT,'Patch notes'\n'Generation'\n'Increased variet...,True,False,False,https://www.nomanssky.com//atlas-rises-update,NaN,NaN
3,next-update,NaT,'Patch notes'\n'Third Person'\n'Third person c...,True,False,False,https://www.nomanssky.com//next-update/,NaN,NaN
4,abyss-update,NaT,'Patch notes'\n'Underwater Gameplay and Visual...,True,False,False,https://www.nomanssky.com//abyss-update/,NaN,NaN
...,...,...,...,...,...,...,...,...,...
363,Expedition Seventeen: Titan,2025-02-12,"'Expedition Seventeen: Titan'\n'February 12, 2...",True,False,False,https://www.nomanssky.com//2025/02/expedition-...,<NA>,17336339.0
364,Worlds Part II - 5.56,2025-02-13,"'Worlds Part II - 5.56'\n'February 13, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,<NA>,17351003.0
365,Worlds Part II - 5.57,2025-02-14,"'Worlds Part II - 5.57'\n'February 14, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,<NA>,17371496.0
366,Worlds Part II - 5.57,2025-02-14,"'Worlds Part II - 5.57'\n'February 14, 2025'\n...",False,True,False,https://www.nomanssky.com//2025/02/worlds-part...,<NA>,17367982.0


In [10]:
combidf.loc[combidf["Date"].isnull()==True]

,Title,Date,Notes,Patch update,Bug fix update,Scrape error,Patch page url,Patch Title,BuildID
0,foundation-update,NaT,'Patch notes'\n'General Gameplay'\n'Fixed tech...,True,False,False,https://www.nomanssky.com//foundation-update,NaN,NaN
1,pathfinder-update,NaT,'Patch Notes'\n'Audio'\n'50% more generative s...,True,False,False,https://www.nomanssky.com//pathfinder-update,NaN,NaN
2,atlas-rises-update,NaT,'Patch notes'\n'Generation'\n'Increased variet...,True,False,False,https://www.nomanssky.com//atlas-rises-update,NaN,NaN
3,next-update,NaT,'Patch notes'\n'Third Person'\n'Third person c...,True,False,False,https://www.nomanssky.com//next-update/,NaN,NaN
4,abyss-update,NaT,'Patch notes'\n'Underwater Gameplay and Visual...,True,False,False,https://www.nomanssky.com//abyss-update/,NaN,NaN
5,visions-update,NaT,'Patch notes'\n'Missions and Progression'\n'Fi...,True,False,False,https://www.nomanssky.com//visions-update/,NaN,NaN
6,beyond-update,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//beyond-update/,NaN,NaN
7,synthesis-update,NaT,'Menu'\n'Latest patch notes'\n'Hello Games Clo...,True,False,False,https://www.nomanssky.com//synthesis-update/,NaN,NaN
8,living-ship-update,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//living-ship-update/,NaN,NaN
9,exo-mech-update,NaT,'Buy now on'\n'More purchase options »'\n'Patc...,True,False,False,https://www.nomanssky.com//exo-mech-update/,NaN,NaN


In [11]:
sdbdf.loc[sdbdf["Patch Title"].isnull()==False]

,Date,Patch Title,BuildID
3,2025-02-12,"MAJOR Expedition 17: ""Titan"" Now Live!",17336353
12,2025-01-29,MAJOR Introducing...Worlds Part II,17171485
28,2024-10-23,MAJOR Introducing..The Cursed Expedition!,16151585
49,2024-05-29,MAJOR The Adrift Expedition is now on!,14533396
86,2023-08-24,Introducing...No Man's Sky Echoes,12022825
106,2023-06-07,The Singularity Expedition Starts Today!,11414214
120,2023-03-08,FRACTAL 4.13,10712241
121,2023-03-03,FRACTAL 4.12.1,10674802
134,2022-07-27,"No Man's Sky 8th Expedition, Polestar, now live!",9194209
199,2020-06-11,Introducing...Cross Play!,5147635


## Conclusions
Need to find alternative data source with which to cross reference named update release dates